In [2]:
using IRTools

In [9]:
function nielson1(x) # a factorial function
    y = x
    z = 1
    while y > 1
        z = z * y
        y = y - 1
    end
    y = 0
end

ir = @code_ir(nielson1(1))
println(ir)
for block in IRTools.blocks(ir)
    println(IRTools.branches(block))
    println(IRTools.successors(block))
    for stmt in block
            println(stmt)
        
    end
end

succ = [ (b1,b2) for b1 in IRTools.blocks(ir) for b2 in IRTools.successors(b1) ]
def =  [ (b,s[1]) for b in IRTools.blocks(ir) for s in b1 ]
def =  [ (b,s[1]) for b in IRTools.blocks(ir) for s in b1 ]
# statements are defined by 

#=

succ(b1, b2)
def(b1, v1) # uniquely defined in branch
input(b1, b1) # input to branch

uses(v1, v2) # if v2 in rhs of v1 def
congruent( v1, v2  ) # if v1 is output of block and v2 is input.



I could also work with surface syntax julia,, which is similar to IMP.
=#


1: (%1, %2)
  br 2 (%2, 1)
2: (%3, %4)
  %5 = %3 > 1
  br 4 unless %5
  br 3
3:
  %6 = %4 * %3
  %7 = %3 - 1
  br 2 (%7, %6)
4:
  return 0
IRTools.Inner.Branch[br 2 (%2, 1)]
IRTools.Inner.Block[2: (%3, %4)
  %5 = %3 > 1
  br 4 unless %5
  br 3]
IRTools.Inner.Branch[br 4 unless %5, br 3]
IRTools.Inner.Block[4:
  return 0, 3:
  %6 = %4 * %3
  %7 = %3 - 1
  br 2 (%7, %6)]
%5 => IRTools.Inner.Statement(:(%3 > 1), Any, 3)
IRTools.Inner.Branch[br 2 (%7, %6)]
IRTools.Inner.Block[2: (%3, %4)
  %5 = %3 > 1
  br 4 unless %5
  br 3]
%6 => IRTools.Inner.Statement(:(%4 * %3), Any, 4)
%7 => IRTools.Inner.Statement(:(%3 - 1), Any, 5)
IRTools.Inner.Branch[return 0]
IRTools.Inner.Block[]


https://blog.rogerluo.me/2019/07/27/yassad/

what if I used the evlator for individual z3 statements. Uhh dispatch might be stripped off at this point


In [4]:
foo(x) = 2 * x



foo (generic function with 1 method)

In [24]:
ir = @code_ir(foo(1))

s1 = collect(ir)[1]

IRTools.evalir(ir, 2)

LoadError: MethodError: no method matching ##254(::Int64)
Closest candidates are:
  ##254(::Any, !Matched::Any) at /home/philip/.julia/packages/IRTools/aSVI5/src/eval.jl:18

In [7]:
IRTools.var(3)
IRTools.Statement( , Any)

%3

In [21]:
foo(x) = x > 3 ? 4 : 5 
ir = @code_ir(foo(1))
IRTools.blocks(ir)

2-element Array{IRTools.Inner.Block,1}:
 1: (%1, %2)
  %3 = %2 > 3
  br 2 unless %3
  return 4
 2:
  return 5

In [46]:
using PyCall
z3 = pyimport("z3")
# z3 datalog
# reaching definitions.
# https://ericpony.github.io/z3py-tutorial/fixpoint-examples.htm
fp = z3.Fixedpoint()
a, b, c = z3.Bools("a b c")

fp.register_relation(a.decl(), b.decl(), c.decl())
fp.rule(a,b)
fp.rule(b,c)
fp.set( engine="datalog") # , generate_explanations=true hmm

println("current set of rules\n", fp)
println(fp.query(a))

fp.fact(c)
println("updated set of rules\n", fp)
println(fp.query(a))
println(fp.get_answer())

# generate_explanations. That's interesting

current set of rules
PyObject (declare-rel b ())
(declare-rel a ())
(declare-rel c ())
(rule (=> (and b) a))
(rule (=> (and c) b))

PyObject unsat
updated set of rules
PyObject (declare-rel b ())
(declare-rel a ())
(declare-rel c ())
(rule (=> b a))
(rule (=> c b))
(rule c)

PyObject sat
PyObject True


In [55]:
using PyCall
z3 = pyimport("z3")
# z3 datalog
# reaching definitions.
# https://ericpony.github.io/z3py-tutorial/fixpoint-examples.htm
fp = z3.Fixedpoint()
s  = z3.FiniteDomainSort("Foo", 10)
#a, b, c = z3.Bools("a b c")
a = z3.Const("a", s)
f = z3.Function("f", s, z3.BoolSort())
fp.register_relation(f)
#fp.rule(a,b)
#fp.rule(b,c)
fp.fact(f(z3.FiniteDomainVal(9, s)))
fp.set( engine="datalog") # , generate_explanations=true hmm

println("current set of rules\n", fp)
#println(fp.query(f(a)))


println("updated set of rules\n", fp)
println(fp.query(f(z3.FiniteDomainVal(9, s))))
println(fp.get_answer())

current set of rules
PyObject (declare-rel f (Foo))
(rule (f 9))

updated set of rules
PyObject (declare-rel f (Foo))
(rule (f 9))

PyObject sat
PyObject True


In [71]:

using PyCall
z3 = pyimport("z3")
# z3 datalog
# reaching definitions.
# https://ericpony.github.io/z3py-tutorial/fixpoint-examples.htm
fp = z3.Fixedpoint()
Color, (red, green, blue) = z3.EnumSort("Color", ["red","green","blue"])
#s  = z3.FiniteDomainSort("Foo", 10)
#a, b, c = z3.Bools("a b c")
a = z3.Const("a", Color)
fp.declare_var(a)
f = z3.Function("f", Color, z3.BoolSort())
fp.register_relation(f)
#fp.rule(a,b)
#fp.rule(b,c)
fp.fact(f(red))
fp.set( engine="datalog") # , generate_explanations=true hmm

println("current set of rules\n", fp)
#println(fp.query(f(a)))


println("updated set of rules\n", fp)
println(fp.query(f(red)))
println(fp.get_answer())

#println(fp.query(f)
println(fp.get_answer())
println(fieldnames(typeof(fp)))

current set of rules
PyObject (declare-datatypes ((Color 0)) (((red) (green) (blue))))
(declare-rel f (Color))
(declare-var A Color)
(rule (f red))

updated set of rules
PyObject (declare-datatypes ((Color 0)) (((red) (green) (blue))))
(declare-rel f (Color))
(declare-var A Color)
(rule (f red))

PyObject sat
PyObject True
PyObject True
(:o,)


So the tutorial talks about a PDR solver. Is this superseded by spacer?

engine (symbol) Select: auto-config, datalog, bmc, spacer (default: auto-config)
How do i store just boring constants?
New Sort?
ADT?

https://specs.openstack.org/openstack/congress-specs/specs/rocky/alternative-engine-z3.html
Hmm. The docs here suggest I asscoaited atoms to bitvectors manually.
Ok I can use FiniteDomainVal.
I guess I could make a named const + a constraint picking which one
And then Z3 does track variable names. That's not so bad.

vars = []
get_var(x, vars) = FiniteDomainVal()

Hmm. I'm not sure the z3 solver gets me dumb datalog.

1. Write raw dog set update equations
2. use souffle
3. maybe z3 does
4. use dataframes? naive or seminaive?


http://webdam.inria.fr/Alice/ foundations of dbs

Souffle does seem cool https://souffle-lang.github.io/examples
But also the bap graph traversal was cool



In [47]:
fp.help()

bmc.linear_unrolling_depth (unsigned int) Maximal level to explore (default: 4294967295)
ctrl_c (bool) enable interrupts from ctrl-c (default: true)
datalog.all_or_nothing_deltas (bool) compile rules so that it is enough for the delta relation in union and widening operations to determine only whether the updated relation was modified or not (default: false)
datalog.check_relation (symbol) name of default relation to check. operations on the default relation will be verified using SMT solving (default: null)
datalog.compile_with_widening (bool) widening will be used to compile recursive rules (default: false)
datalog.dbg_fpr_nonempty_relation_signature (bool) if true, finite_product_relation will attempt to avoid creating inner relation with empty signature by putting in half of the table columns, if it would have been empty otherwise (default: false)
datalog.default_relation (symbol) default relation implementation: external_relation, pentagon (default: pentagon)
datalog.default_table

In [36]:
function nielson1(x) # a factorial function
    y = x
    z = 1
    while y > 1
        z = z * y
        y = y - 1
    end
    y = 0
end
ir = @code_ir(nielson1(1))

1: (%1, %2)
  br 2 (%2, 1)
2: (%3, %4)
  %5 = %3 > 1
  br 4 unless %5
  br 3
3:
  %6 = %4 * %3
  %7 = %3 - 1
  br 2 (%7, %6)
4:
  return 0

# Good simple example functions
factorial
array sum
sumn

reaching definition  https://en.wikipedia.org/wiki/Reaching_definition
a relation between variables and operations/lines of code
I guess anything with a lhs.





Doing some datalog over the IRTools graph sounds kind of cool.
I could try to use z3py maybe

Reify x == y

https://stackoverflow.com/questions/38601141/what-is-the-difference-between-and-comparison-operators-in-julia

One might be able to run something like the boehm garbage collector to get something like the state of the heap.

Valgrind also maybe?


In [30]:
module Play

mutable struct LL
    head::Int64
    tail
end

cons(x,y) = LL(x,y)

x = cons(1, nothing)
y = cons(2,x)
z = cons(3,x)
x2 = cons(1, nothing)
w = cons(2,x2)

println(y === w)
println(x === x2)
println(y.tail === z.tail)
println(y.tail === w.tail)

println(fieldnames(typeof(y)))
println(fieldnames(typeof(y)))
#=
t = typeof(x)
for fname in fieldnames(t)
    
end
=#

d = Dict()
d[y.tail] = 1
d[z.tail] = 2
d[w.tail] = 3
println(d)


#=

RBtreenode
others?

refcell union find
=#

end

false
false
true
false
(:head, :tail)
(:head, :tail)


Dict{Any,Any}(Main.Play.LL(1, nothing) => 3,Main.Play.LL(1, nothing) => 2)


Main.Play